In [1]:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

     ------------------------------------- 797.3/797.3 kB 12.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 2.0/2.0 MB 31.4 MB/s eta 0:00:00
     ------------------------------------- 192.1/192.1 kB 12.1 MB/s eta 0:00:00
  Created wheel for ibm_db: filename=ibm_db-3.1.0-py3-none-any.whl size=27687186 sha256=b751ce300cf67dd2e50c490ee183cca6abd45def36d5b3a28f22981016f773c1
  Stored in directory: c:\users\bhaumik patel\appdata\local\pip\cache\

In [14]:
import csv, sqlite3

con = sqlite3.connect("ChicagoCrimeData.db")
cur = con.cursor()

In [17]:

!pip install pandas==1.1.5

In [19]:
%load_ext sql

In [20]:
%sql sqlite:///ChicagoCrimeData.db

In [22]:
import pandas
df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

In [23]:
%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_CRIME_DATA');

 * sqlite:///ChicagoCrimeData.db
Done.


name,type,length(type)
ID,INTEGER,7
CASE_NUMBER,TEXT,4
DATE,TEXT,4
BLOCK,TEXT,4
IUCR,TEXT,4
PRIMARY_TYPE,TEXT,4
DESCRIPTION,TEXT,4
LOCATION_DESCRIPTION,TEXT,4
ARREST,INTEGER,7
DOMESTIC,INTEGER,7


# Problem 1: Find the total number of crimes recorded in the CRIME table.

In [25]:
%sql select count(*) from CHICAGO_CRIME_DATA

 * sqlite:///ChicagoCrimeData.db
Done.


count(*)
533


# Problem 2: List community areas with per capita income less than 11000.

In [26]:
%sql select community_area_name from CENSUS_DATA where per_capita_income < 11000

 * sqlite:///ChicagoCrimeData.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


# Problem 3: List all case numbers for crimes involving minors?

In [27]:
%sql select case_number from CHICAGO_CRIME_DATA where description like '%MINOR%'

 * sqlite:///ChicagoCrimeData.db
Done.


CASE_NUMBER
HL266884
HK238408


# Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [28]:
%sql select * from CHICAGO_CRIME_DATA where primary_type like '%KIDNAP%' and description like '%CHILD%'

 * sqlite:///ChicagoCrimeData.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


# Problem 5: What kind of crimes were recorded at schools?

In [29]:
%sql select distinct(primary_type) from CHICAGO_CRIME_DATA where location_description like 'SCHOOL%'

 * sqlite:///ChicagoCrimeData.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


# Problem 6: List the average safety score for all types of schools.

In [58]:
%%sql select 'Elementary, Middle, or High School', avg(safety_score)
from CHICAGO_PUBLIC_SCHOOLS_DATA
group by 'Elementary, Middle, or High School'

 * sqlite:///ChicagoCrimeData.db
Done.


"'Elementary, Middle, or High School'",avg(safety_score)
"Elementary, Middle, or High School",49.50487329434698


# Problem 7: List 5 community areas with highest % of households below poverty line.

In [43]:
%sql select community_area_name, percent_households_below_poverty from census_data order by percent_households_below_poverty desc limit 5

 * sqlite:///ChicagoCrimeData.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


# Problem 8: Which community area(number) is most crime prone?

In [46]:
%sql select community_area_number, count(*) as frequency from CHICAGO_CRIME_DATA group by community_area_number order by frequency desc limit 1

 * sqlite:///ChicagoCrimeData.db
Done.


COMMUNITY_AREA_NUMBER,frequency
25.0,43


# Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [47]:
%sql select community_area_name from CENSUS_DATA where hardship_index = (Select max(hardship_index) from CENSUS_DATA)

 * sqlite:///ChicagoCrimeData.db
Done.


COMMUNITY_AREA_NAME
Riverdale


# Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [48]:
%sql select community_area_name from CENSUS_DATA where community_area_number = (select community_area_number from CHICAGO_CRIME_DATA where community_area_number = 25 limit 1)

 * sqlite:///ChicagoCrimeData.db
Done.


COMMUNITY_AREA_NAME
Austin
